# Model Development - Solar CAPEX Estimator

This notebook develops a machine learning model to predict total installed costs (CAPEX) for commercial solar installations using the LBNL Tracking the Sun dataset. It has five main sections to guide the process from data loading to model evaluation, which will eventually be used in a SolarCapexEstimator class. The five sections are:

1) **DataLoader**, responsible for loading Tracking the Sun data from CSV files and filtering it to rows relevant to our use case (commercial solar installations in the US).

2) **DataCleaner**, responsible for cleaning the data by removing rows with missing or invalid values in the target column (total installed price). It also drops columns where most values are missing or fills in missing values with appropriate strategies.

3) **FeatureEngineer**, responsible for creating new features from the existing data that may help the model learn better, but are still readable and interpretable by users.

4) **ModelTrainer**, responsible for training a machine learning model (e.g. linear regression, random forest, or gradient boosting) on the cleaned and feature-engineered data.

5) **ModelEvaluator**, responsible for evaluating the trained model's performance using appropriate metrics and validation techniques.

Using composition (separate classes coordinated by a higher-level estimator) keeps each step—data loading, cleaning, feature engineering, training, and evaluation—focused on a single responsibility. This improves modularity, testability, and reuse: each component can be developed, swapped, or improved independently without affecting the rest of the pipeline.

## Setup and Imports

Import necessary libraries for data manipulation, visualization, and modeling.

In [11]:
import pandas as pd
import numpy as np
from plotly import graph_objects as go
from pathlib import Path
from typing import Optional, List

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## 1. DataLoader - Loading TTS Data

We'll start by using our **`DataLoader`** class to load the Tracking the Sun dataset. This class handles the messy details of reading CSV files and filtering them to just the records we care about.

The `DataLoader` class provides:
- Automatic handling of multiple CSV files in a directory
- Date parsing for installation dates
- Year-based filtering (we want 2019-2023)
- Customer segment filtering (commercial and non-residential only)

In [ ]:
class DataLoader:
    """
    Data loader for LBNL Tracking the Sun dataset.

    This class handles TTS-specific data loading, cleaning, and filtering
    operations independent of the modeling pipeline.

    Parameters
    ----------
    tts_data_directory : str
        Path to the directory containing the raw TTS data files.

    Attributes
    ----------
    raw_data_directory: Path
        Directory containing the raw TTS data files.

    """

    def __init__(self, tts_data_directory: str):
        self.tts_data_directory = Path(tts_data_directory)
        self.df = None

        self.valid_customer_segments = ['COM', 'RES_MF', 'RES_SF', 'RES', 'AGRICULTURAL',
       'OTHER TAX-EXEMPT', 'GOV', 'SCHOOL', 'NON-RES', 'NON-PROFIT']

    def _filter_by_years(self, df, year_min=None, year_max=None):
        """
        Filter data to specific installation years.

        Parameters
        ----------
        df : pd.DataFrame
            Dataframe to filter.
        year_min : int, optional
            Minimum installation year to include. If None, includes all years.
        year_max : int, optional
            Maximum installation year to include. If None, includes all years.

        Returns
        -------
        pd.DataFrame
            Filtered dataframe.

        Raises
        ------
        ValueError
            If dataframe has not been loaded.
        """
        if df is None:
            raise ValueError("Data not loaded. Call load_raw() first.")

        if year_min is not None:
            df = df[df.installation_date.dt.year >= year_min]
        if year_max is not None:
            df = df[df.installation_date.dt.year <= year_max]

        return df

    def _filter_by_customer_segment(self, df, segments):
        """
        Filter data to specific customer segments.

        Parameters
        ----------
        segments : list of str
            Customer segments to include (e.g., ['COM', 'NON-RES']).

        Returns
        -------
        pd.DataFrame
            Filtered dataframe.
        segments : list of str
            Customer segments to filter to. If None, includes all segments.

        Raises
        ------
        ValueError
            If dataframe has not been loaded.
        """
        if df is None:
            raise ValueError("Data not loaded. Call load_raw() first.")

        df = df[df['customer_segment'].isin(segments)]

        return df
    
    def _validate_filters(self, year_min, year_max, customer_segments):
        """
        Validate filter parameters.

        Parameters
        ----------
        year_min : int, optional
            Minimum installation year to include. If None, includes all years.
        year_max : int, optional
            Maximum installation year to include. If None, includes all years.
        customer_segments : list of str, optional
            Customer segments to include (e.g., ['COM', 'NON-RES']).

        Raises
        ------
        ValueError
            If year_min is greater than year_max or if customer_segments is not a list of strings.
        """
        if year_min is not None and year_max is not None and year_min > year_max:
            raise ValueError("year_min cannot be greater than year_max.")
        
        if customer_segments is not None:
            if not isinstance(customer_segments, list) or not all(isinstance(seg, str) for seg in customer_segments):
                raise ValueError("customer_segments must be a list of strings.")
            if not set(customer_segments).issubset(set(self.valid_customer_segments)):
                raise ValueError(f"customer_segments must be a subset of {self.valid_customer_segments}.")
            

    def load(
        self,
        year_min: Optional[int] = None,
        year_max: Optional[int] = None,
        customer_segments: Optional[List[str]] = None
    ):
        """
        Load and filter TTS data with common preprocessing steps.

        Parameters
        ----------
        year_min : int, optional
            Minimum year to filter to. If None, includes all years.
        year_max : int, optional
            Maximum year to filter to. If None, includes all years.
        customer_segments : list of str, optional
            Customer segments to filter to. If None, includes all segments.

        Returns
        -------
        pd.DataFrame
            Filtered and cleaned dataframe.
        """

        csvs = list(self.tts_data_directory.glob('*.csv'))

        self._validate_filters(year_min, year_max, customer_segments)

        if csvs:
            self.df = pd.DataFrame()
            for csv in csvs:
                csv_df = pd.read_csv(csv, parse_dates=['installation_date'])

                if year_min is not None or year_max is not None:
                    csv_df = self._filter_by_years(csv_df, year_min, year_max)

                if customer_segments is not None:
                    csv_df = self._filter_by_customer_segment(csv_df, customer_segments)

                self.df = pd.concat([self.df, csv_df], ignore_index=True)
                print(f"Loaded {len(csv_df)} rows from {csv.name}")
        
        else:
            raise ValueError(f"No CSV files found in directory {self.tts_data_directory}")


    def get_data(self):
        """
        Get the current dataframe.

        Returns
        -------
        pd.DataFrame
            Current dataframe.

        Raises
        ------
        ValueError
            If dataframe has not been loaded.
        """
        if self.df is None:
            raise ValueError("Data not loaded. Call load_raw() or load() first.")

        return self.df

### Instantiate and Load Data

Now we'll create a `DataLoader` instance pointing to our raw data directory and use it to load commercial solar installations from 2019-2023.

In [10]:
tts_dataloader = DataLoader(tts_data_directory='../data/raw')

tts_dataloader.load(year_min=2019, year_max=2023, customer_segments=['COM', 'NON-RES'])

/var/folders/zj/gmwqw_hj5bj1f90myf__wr_c0000gn/T/ipykernel_38577/173871540.py:146: DtypeWarning: Columns (1,2,3,11,15,16,18,28,29,31,32,34,35,38,39,40,53,54,56,57,59,60,74,75,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_df = pd.read_csv(csv, parse_dates=['installation_date'])


Loaded 22118 rows from TTS_LBNL_public_file_29-Sep-2025_all.csv


### What We Loaded

We successfully loaded 22,118 rows of commercial and non-residential solar installation data from 2019-2023. Each row represents one solar installation project with details about system size, location, components, and cost.